In [4]:
import pandas as pd
import EDA
from tqdm import tqdm
from EDA import eda
import random
import nltk
nltk.download('wordnet')
nltk.download('omw')

[nltk_data] Downloading package wordnet to /home/hu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/hu/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.


True

In [26]:
from nltk.corpus import wordnet
synonyms = set()
word = 'shine'
if wordnet.synsets(word):
    for syn in wordnet.synsets(word):
        for l in syn.lemma_names('spa'):
            synonym = l.replace('_', " ").lower()
            synonym = "".join([char for char in synonym if (char.isalpha() or char == ' ')])
            synonyms.add(synonym)
if word in synonyms:
    synonyms.remove(word)
print(synonyms)

{'relumbrar', 'resplandor', 'lucir', 'relucir', 'reflejar', 'resplandecer', 'brillo', 'esplendor', 'brillar', 'radiancia espectral'}


In [2]:
eda('trim so that we have the desired number of augmented sentences')

['trim so that have desired number of augmented sentences',
 'trim so that we have the desired number of want augmented sentences',
 'trim so that we have the desired number of augmented sentences',
 'and so trim so that we have the desired number of augmented sentences']

In [3]:
df = pd.read_json('./cleaned_labeled_data_v3.json')

In [4]:
df.head(10)

,category,headline,description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,"""It is not right to equate horrific incidents ..."
6,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This ...,There's a great mini-series joining this week.
7,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,"Myer's kids may be pushing for a new ""Powers"" ..."
8,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,You're getting a recent Academy Award-winning ...
9,ENTERTAINMENT,Justin Timberlake Visits Texas School Shooting...,"The pop star also wore a ""Santa Fe Strong"" shi..."


In [5]:
content = df['description']
headline = df['headline']
category = df['category']

In [6]:
category_dic = {}
for idx,cate in enumerate(list(set(df['category']))):
    category_dic[cate] = idx

In [7]:
category_dic

{'SPORTS': 0,
 'WELLNESS': 1,
 'TRAVEL': 2,
 'DIVORCE': 3,
 'ENVIRONMENT': 4,
 'MEDIA': 5,
 'WOMEN': 6,
 'GROUPS VOICES': 7,
 'RELIGION': 8,
 'FOOD & DRINK': 9,
 'CRIME': 10,
 'WEDDINGS': 11,
 'IMPACT': 12,
 'SCIENCE & TECH': 13,
 'WORLD NEWS': 14,
 'WEIRD NEWS': 15,
 'STYLE & BEAUTY': 16,
 'HOME & LIVING': 17,
 'ARTS & CULTURE': 18,
 'MISCELLANEOUS': 19,
 'ENTERTAINMENT': 20,
 'COMEDY': 21,
 'EDUCATION': 22,
 'PARENTING': 23,
 'POLITICS': 24,
 'BUSINESS & FINANCES': 25}

In [8]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in category if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

SPORTS:3703
WELLNESS:22639
TRAVEL:9218
DIVORCE:3406
ENVIRONMENT:3309
MEDIA:2174
WOMEN:2871
GROUPS VOICES:10242
RELIGION:1798
FOOD & DRINK:6816
CRIME:2579
WEDDINGS:3632
IMPACT:3032
SCIENCE & TECH:3534
WORLD NEWS:6842
WEIRD NEWS:1679
STYLE & BEAUTY:10541
HOME & LIVING:3946
ARTS & CULTURE:3095
MISCELLANEOUS:1962
ENTERTAINMENT:11626
COMEDY:3734
EDUCATION:1784
PARENTING:11804
POLITICS:27829
BUSINESS & FINANCES:6654
total: 170449
max_cate:27829
min_cate:1679
[3703, 22639, 9218, 3406, 3309, 2174, 2871, 10242, 1798, 6816, 2579, 3632, 3032, 3534, 6842, 1679, 10541, 3946, 3095, 1962, 11626, 3734, 1784, 11804, 27829, 6654]


#### find a proper increase num_aug

In [9]:
for ca in category_dic.keys():
    p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
    print(category_dic[ca],dic_count[category_dic[ca]],p)

0 3703 7.081534722366481
1 22639 1.158307481643318
2 9218 2.844751906804413
3 3406 7.69903789692398
4 3309 7.924727433340308
5 2174 12.0620621328993
6 2871 9.133724513034858
7 10242 2.5603322668349033
8 1798 14.58449559339437
9 6816 3.8472598410978693
10 2579 10.167864706057804
11 3632 7.219967807522873
12 3032 8.64872133143901
13 3534 7.420181968569066
14 6842 3.832640028781508
15 1679 15.618179319191828
16 10541 2.4877073405676007
17 3946 6.645444266833015
18 3095 8.472673045855599
19 1962 13.365404218615229
20 11626 2.2555412933875
21 3734 7.022743191463063
22 1784 14.698947913073473
23 11804 2.221528556160884
24 27829 0.942287652338319
25 6654 3.9409262213590437


In [10]:
augumented_news = [[],[],[]]

In [11]:
for cate,head,cont in tqdm(zip(category,headline,content)):
    if max(len(head),len(cont)>10):
        p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
        if len(head)>len(cont):
            new_heads = eda(head,num_aug=max(6,int(p)))
            for new in new_heads:
                augumented_news[0].append(cate)
                augumented_news[1].append(new)
                augumented_news[2].append(cont)
        else:
            new_conts = eda(cont)
            for new in new_conts:
                augumented_news[0].append(cate)
                augumented_news[1].append(head)
                augumented_news[2].append(new)
    else:
        augumented_news[0].append(cate)
        augumented_news[1].append(head)
        augumented_news[2].append(cont)

170449it [02:02, 1389.24it/s]


In [12]:
len(augumented_news[0])

851194

In [13]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in augumented_news[0] if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

SPORTS:19267
WELLNESS:108798
TRAVEL:44678
DIVORCE:15915
ENVIRONMENT:15951
MEDIA:11181
WOMEN:14564
GROUPS VOICES:51660
RELIGION:9008
FOOD & DRINK:34890
CRIME:13016
WEDDINGS:16817
IMPACT:15105
SCIENCE & TECH:17268
WORLD NEWS:34647
WEIRD NEWS:9186
STYLE & BEAUTY:50058
HOME & LIVING:19701
ARTS & CULTURE:15529
MISCELLANEOUS:10161
ENTERTAINMENT:61886
COMEDY:19655
EDUCATION:8903
PARENTING:57463
POLITICS:143750
BUSINESS & FINANCES:32137
total: 851194
max_cate:143750
min_cate:8903
[19267, 108798, 44678, 15915, 15951, 11181, 14564, 51660, 9008, 34890, 13016, 16817, 15105, 17268, 34647, 9186, 50058, 19701, 15529, 10161, 61886, 19655, 8903, 57463, 143750, 32137]


In [14]:
shuffle_idx = [i for i in range(0,len(augumented_news[0]))]
random.shuffle(shuffle_idx)
shuffled_data = [[],[],[]]
for idx in shuffle_idx:
    shuffled_data[0].append(augumented_news[0][idx])
    shuffled_data[1].append(augumented_news[1][idx])
    shuffled_data[2].append(augumented_news[2][idx])
    

In [15]:
augmented_df = {
    'category':shuffled_data[0],
    'head':shuffled_data[1],
    'content':shuffled_data[2]
}

In [16]:
augmented_df = pd.DataFrame(data=augmented_df)

In [17]:
augmented_df['category'].head(10)

0       WEIRD NEWS
1     FOOD & DRINK
2           SPORTS
3         WEDDINGS
4    MISCELLANEOUS
5          DIVORCE
6           COMEDY
7    GROUPS VOICES
8     FOOD & DRINK
9       WEIRD NEWS
Name: category, dtype: object

In [18]:
augmented_df.to_json('./augmented_labeled_news.json')